### Libraries

In [146]:
import pandas as pd

import folium
from folium.plugins import HeatMap
from folium import plugins

### San Francisco's Latitude and Longitude

In [84]:
lat = 37.773972
lon = -122.431297

### Plot a Map of San Francisco

In [85]:
sf = folium.Map([lat, lon], tiles='Stamen Terrain', zoom_start=13)
display(sf)

### San Francisco's City Hall (Lat and Lon)

In [86]:
ch_lat = 37.77919
ch_lon = -122.41914

### San Francisco's City Hall Marker

In [87]:
folium.Marker([ch_lat, ch_lon], popup='London Bridge').add_to(sf)
display(sf)

### Import the Dataset

In [88]:
data = pd.read_csv('Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')

### Settings

In [89]:
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 35)

### DRUG/NARCOTIC Dataset

In [90]:
drug_data = data[data['Category'].isin(['DRUG/NARCOTIC'])].reset_index(drop=True)
drug_data['M/Y'] = drug_data['Date'].str.strip().str[:3] + drug_data['Date'].str.strip().str[-4:]
drug_data = drug_data[drug_data['M/Y'].isin(['06/2016','07/2016'])].reset_index(drop=True)

### Draw a Dot for Each Arrest

In [91]:
for i in range(len(drug_data)):
    folium.CircleMarker(location=[drug_data['Y'][i], drug_data['X'][i]], radius=2, weight=2, color='red').add_to(sf)

display(sf)

### SEX OFFENSES, NON FORCIBLE Dataset

In [92]:
so_nf = data[data['Category'].isin(['SEX OFFENSES, NON FORCIBLE'])].reset_index(drop=True)

### List comprehension to make out list of lists

In [93]:
temp_data = so_nf[['Y', 'X']]
temp_data = temp_data.dropna(axis=0, subset=['Y','X'])
heat_data = [[row['Y'],row['X']] for index, row in temp_data.iterrows()]

### Heatmap

In [123]:
sf = folium.Map([lat, lon], tiles='Stamen Toner', zoom_start=13)
HeatMap(heat_data, min_opacity=0.5, max_zoom=9, radius=20, blur=12).add_to(sf)
display(sf)

### Creating the Weekhour Column

In [129]:
data['int_hour'] = pd.to_datetime(data['Time'], format = '%H:%M').dt.hour
data['int_day'] = data['DayOfWeek'].str.replace('Monday', '0').replace('Tuesday', '24').replace('Wednesday', '48').str.replace('Thursday', '72').str.replace('Friday', '96').str.replace('Saturday', '120').str.replace('Sunday', '144')
data['int_day'] = data['int_day'].astype(int)
data['Weekhour'] = data['int_hour'] + data['int_day']

### PROSTITUTE Dataset

In [133]:
prostitution = data[data['Category'].isin(['PROSTITUTION'])].reset_index(drop=True)

### List comprehension to make out list of lists

In [150]:
temp_data = prostitution[['Y', 'X','Weekhour']]
temp_data = temp_data.dropna(axis=0, subset=['Y','X','Weekhour'])
heat_data = [[[row['Y'],row['X']] for index, row in temp_data[temp_data['Weekhour'] == i].iterrows()] for i in range(0,166)]

In [151]:
sf = folium.Map([lat, lon], tiles='Stamen Toner', zoom_start=13)
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(sf)

display(sf)